In [1]:
import pandas as pd
#from mlxtend.preprocessing import TransactionEncoder
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)


In [3]:
data_low = pd.read_csv('C:/Users/Lee Dongjin/Desktop/Gusto Mate/beginner_recipe.csv')
data_mid = pd.read_csv('C:/Users/Lee Dongjin/Desktop/Gusto Mate/intermediate_recipe.csv')
data = pd.concat([data_low, data_mid], ignore_index=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   메뉴 이름   1572 non-null   object 
 1   방법 분류   1572 non-null   object 
 2   국가 분류   1572 non-null   object 
 3   테마 분류   1572 non-null   object 
 4   난이도 분류  1572 non-null   object 
 5   칼로리     1572 non-null   float64
 6   조회수     1572 non-null   int64  
 7   분량      1572 non-null   int64  
 8   주재료 이름  1572 non-null   object 
 9   부재료 이름  1564 non-null   object 
 10  양념      1288 non-null   object 
 11  레시피     1572 non-null   object 
 12  조리시간    1572 non-null   int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 159.8+ KB


In [5]:
data.head()

,메뉴 이름,방법 분류,국가 분류,테마 분류,난이도 분류,칼로리,조회수,분량,주재료 이름,부재료 이름,양념,레시피,조리시간
0,견과류 영양솥밥,"['밥', '죽', '스프']",한식,메인요리,쉬움,311.0,28441,4,"['백미 1과3/4컵', '은행 12개', '밤 8개', '대추 4개', '잣 1큰술']",['물 1과3/4컵'],NaN,"['1. 쌀을 씻어 30분 정도 물에 불려둔다.', '2. 은행은 볶아 껍질을 벗긴다.', '3. 밤은 껍질을 벗기고 대추는 깨끗이 씻어둔다.', '4. 쌀과 밤을 넣고 밥을 짓다가 한소끔 끓어오르면 대추, 은행, 잣을 넣고 약한 불로 뜸을 들인다.', '레시피 출처 : 월간 헬스조선 / 대추나 잣, 호두 등의 견과류는 오랫동안 익힐 필요가 없다. 대추는 무르거나 퍼질 수 있으므로 밥을 다 한 뒤 뜸들일 때 넣는다.']",60
1,굴밥,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,320.0,145639,2,"['굴 300g', '쌀 200g', '보리쌀 80g']","['어묵(찐것) 50g', '표고버섯 3장', '당근 50g', '완두콩(통조림) 30g', '물 1과1/2컵']",['소금 1작은술'],"['1. 쌀과 보리쌀을 깨끗이 씻어 물에 30분 동안 담가둔다.', '2. 굴은 소금물에 흔들어 씻은 뒤 물기를 뺀다. 찐 어묵은 0.2cm 두께로 썬다.', '3. 마른 표고버섯은 따뜻한 물에 불려 밑동을 없애고 물기를 꼭 짠 다음 잘게 다진다.', '4. 당근은 잘게 다진다.', '5. 쌀과 보리쌀은 깨끗이 씻어 물에 30분 간 불린다.', '6. 불린 쌀과 보리 쌀은 체에 밭쳐 물기를 뺀다.', '7. 냄비에 불린 쌀과 보리쌀, 굴, 찐어묵, 표고버섯, 당근, 완두콩, 소금을 넣어 섞은뒤 물 1과 1/2컵을 넣어 밥을 한다.']",30
2,김치볶음밥,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,560.0,337639,2,"['김치(잘게썬것) 1/3포기', '밥 2공기']","['감자 1/4개', '양파 1/2개', '햄 60g', '식용유 약간', '계란 2개']","['소금 약간', '후춧가루 약간']","['1. 김치는 1cm 크기로 썰고 햄, 양파, 감자도 1cm 정도의 크기로 잘게 썬다.', '2. 프라이팬에 기름을 두르고 김치와 감자를 볶다가 햄과 양파를 넣고 볶아준다.', '3. 양파가 어느 정도 익으면 밥을 넣고 펴듯이 볶아준다.', '4. 소금과 후춧가루로 간을 맞추고 계란프라이를 얹어 낸다.']",20
3,꽃상추쌈,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,352.0,103833,2,"['상추 4장', '밥 1그릇']","['다진쇠고기 100g', '다진양파 1/2개']","['간장 1큰술', '설탕 1/2큰술', '다진파 약간', '다진마늘 약간', '깨소금 약간', '참기름 약간', '고추장 약간']","['1. 쇠고기, 양파는 다져서 준비한다.(양파는 다진 후 물기를 제거한다)', '2. ①의 재료에 불고기 양념(간장 1/2큰술, 설탕 1/4큰술, 참기름, 깨소금, 다진 마늘, 다진 파)을 약하게 하여 고슬고슬하게 볶는다.', '3. 뜨거운 밥을 준비하여 ②의 재료와 깨소금등을 넣어 양념한다.', '4. 상추는 한 입 크기 정도로 손으로 찢어 준비한다.', '5. 밥이 어느정도 식으면 한 입 크기로 찢어놓은 상추에 밥을 동글게 빚어 그 위에 올려놓는다.', '6. 먹기 직전에 고추장을 약간 뿌린다.']",30
4,녹두죽,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,280.0,66164,2,"['녹두 1컵', '쌀 2컵']",['물 14컵'],"['소금 약간', '설탕 약간']","['1. 녹두는 깨끗이 씻어 일어 건진 다음 물을 붓고 푹 삶아 체에 걸러 놓는다.', '2. 쌀은 깨끗이 씻어 물에 불려 건져 놓는다.', '3. 냄비에 녹두의 윗 물만 덜어 내어 붓고 쌀을 끓여 쌀알이 어느 정도 퍼지게 되면 녹두 앙금을 넣어 섞으면서 다시 한 번 끓인다. 녹두 앙금은 나중에 넣어 끓인다.', '4. 쌀알이 잘 퍼지면 그릇에 담아 대추채를 고명으로 띄워내고, 기호에 따라 소금이나 설탕을 넣어서 먹는다.']",60


In [6]:
units = ['컵', '개', '큰술', '작은술', 'g', 'kg', 'ml', 'L', 'cc', 'cm', 'ea', '와', '과', '/', '약간']
unit_pattern = '|'.join(units)
pattern = re.compile(rf'\d+|{unit_pattern}|\[.*?\]|\(.*?\)|[^가-힣\s]')

def extract_ingredients(column_data):
    ingredients = []
    
    for item in column_data:
        if pd.notna(item):
            # 콤마로 분리한 후 각 항목에서 숫자, 괄호 등을 제거
            items = item.strip('[]').split(',')
            for sub_item in items:
                # 숫자, 단위, 괄호 및 기호 등을 제거
                cleaned_item = re.sub(pattern, '', sub_item).strip()
                # 첫 번째 단어 추출
                cleaned_item = cleaned_item.split()[0] if cleaned_item else ''
                if cleaned_item:
                    ingredients.append(cleaned_item)
    return ingredients

def combine_lists(row):
    combined = []
    for col in ['main_ingredients', 'sub_ingredients', 'seasoning']:
        combined.extend(row[col])
    return combined


In [7]:
data['main_ingredients'] = data['주재료 이름'].apply(lambda x: extract_ingredients([x]))
data['sub_ingredients'] = data['부재료 이름'].apply(lambda x: extract_ingredients([x]))
data['seasoning'] = data['양념'].apply(lambda x: extract_ingredients([x]))

# 새로운 칼럼에 합쳐진 리스트 저장
data['whole_ingredients'] = data.apply(combine_lists, axis=1)
data['whole_ingredients']

0                                           [백미, 은행, 밤, 대추, 잣, 물]
1                           [굴, 쌀, 보리쌀, 어묵, 표고버섯, 당근, 완두콩, 물, 소금]
2                           [김치, 밥, 감자, 양파, 햄, 식용유, 계란, 소금, 후춧가루]
3          [상추, 밥, 다진쇠고기, 다진양파, 간장, 설탕, 다진파, 다진마늘, 깨소금, 참기름, 고추장]
4                                              [녹두, 쌀, 물, 소금, 설탕]
                                  ...                            
1567    [감자, 치즈, 찬밥, 하이포크팜햄, 방울토마토, 피망, 식용유, 양파, 토마토케찹, 소금, 후춧가루]
1568                         [가래떡, 베이컨, 옥수수, 피망, 양파, 피자치즈, 토마토케첩]
1569                         [감자, 베이컨, 방울토마토, 프레시파머산치즈, 소금, 후춧가루]
1570              [쌀국수, 바지락, 파프리카, 숙주, 식용유, 다진마늘, 말린고추, 해선장, 굴소스]
1571             [쇠고기, 양송이, 피망, 양파, 당근, 케첩, 우스터소스, 설탕, 미림, 파슬리가루]
Name: whole_ingredients, Length: 1572, dtype: object

## 칼럼 만들기

In [8]:
def calculate_spiciness(row):
    score = 0
    spicy_ingredients = ['매운', '고추', '김치', '고춧가루', '얼큰']
    
    # 메뉴 이름에서 맵기 성분 찾기
    if isinstance(row['메뉴 이름'], str):
        for spicy in spicy_ingredients:
            if spicy in row['메뉴 이름']:
                score += 3

    # 주재료 이름에서 맵기 성분 찾기
    if isinstance(row['주재료 이름'], str):
        row['주재료 이름'] = eval(row['주재료 이름'])
    
    if isinstance(row['주재료 이름'], list):
        for ingredient in row['주재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 1
                
    # 부재료 이름에서 맵기 성분 찾기
    if isinstance(row['부재료 이름'], str):
        row['부재료 이름'] = eval(row['부재료 이름'])
        
    if isinstance(row['부재료 이름'], list):
        for ingredient in row['부재료 이름']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 1
    
    # 양념에서 맵기 성분 찾기
    if isinstance(row['양념'], str):
        row['양념'] = eval(row['양념'])
    
    if isinstance(row['양념'], list):
        for ingredient in row['양념']:
            if any(spicy in ingredient for spicy in spicy_ingredients):
                score += 1
                
    return score

data['spiciness'] = data.apply(calculate_spiciness, axis=1)
   

In [9]:
def is_low_calorie(calorie):
    return 1 if calorie <= 500 else 0

data['low_calorie'] = data['칼로리'].apply(is_low_calorie)

In [10]:
difficulty_mapping = {
    '매우 쉬움': 1,
    '쉬움': 2,
    '보통': 3,
    '어려움': 4
}

# 매핑 적용
data['difficulty'] = data['난이도 분류'].map(difficulty_mapping)

In [11]:
data_add_dummies = pd.get_dummies(data, columns=['국가 분류'])
data_add_dummies = pd.get_dummies(data_add_dummies, columns=['테마 분류'])
data_add_dummies = data_add_dummies.drop(['방법 분류', '조회수', '분량', '주재료 이름', '부재료 이름', '양념', '레시피'], axis = 1)
data_add_dummies.rename(columns={'국가 분류_동남아시아':'is_southeast_asian', 
                                 '국가 분류_멕시코':'is_mexican', 
                                 '국가 분류_서양':'is_western', 
                                 '국가 분류_이탈리아':'is_italian', 
                                 '국가 분류_일본':'is_japanese',
                                 '국가 분류_중국':'is_chinese',
                                 '국가 분류_퓨전':'is_mixed',
                                 '국가 분류_한식':'is_korean',
                                 '테마 분류_간식':'is_snack',
                                 '테마 분류_메인요리':'is_main',
                                 '테마 분류_반찬':'is_side'                            
                                 },
                        inplace=True)


In [12]:
data_add_dummies.head()

,메뉴 이름,난이도 분류,칼로리,조리시간,main_ingredients,sub_ingredients,seasoning,whole_ingredients,spiciness,low_calorie,difficulty,is_southeast_asian,is_mexican,is_western,is_italian,is_japanese,is_chinese,is_mixed,is_korean,is_snack,is_main,is_side
0,견과류 영양솥밥,쉬움,311.0,60,"[백미, 은행, 밤, 대추, 잣]",[물],[],"[백미, 은행, 밤, 대추, 잣, 물]",0,1,2,False,False,False,False,False,False,False,True,False,True,False
1,굴밥,매우 쉬움,320.0,30,"[굴, 쌀, 보리쌀]","[어묵, 표고버섯, 당근, 완두콩, 물]",[소금],"[굴, 쌀, 보리쌀, 어묵, 표고버섯, 당근, 완두콩, 물, 소금]",0,1,1,False,False,False,False,False,False,False,True,False,True,False
2,김치볶음밥,매우 쉬움,560.0,20,"[김치, 밥]","[감자, 양파, 햄, 식용유, 계란]","[소금, 후춧가루]","[김치, 밥, 감자, 양파, 햄, 식용유, 계란, 소금, 후춧가루]",4,0,1,False,False,False,False,False,False,False,True,False,True,False
3,꽃상추쌈,매우 쉬움,352.0,30,"[상추, 밥]","[다진쇠고기, 다진양파]","[간장, 설탕, 다진파, 다진마늘, 깨소금, 참기름, 고추장]","[상추, 밥, 다진쇠고기, 다진양파, 간장, 설탕, 다진파, 다진마늘, 깨소금, 참기름, 고추장]",1,1,1,False,False,False,False,False,False,False,True,False,True,False
4,녹두죽,매우 쉬움,280.0,60,"[녹두, 쌀]",[물],"[소금, 설탕]","[녹두, 쌀, 물, 소금, 설탕]",0,1,1,False,False,False,False,False,False,False,True,False,True,False


In [13]:
pd.unique(data_add_dummies['spiciness'])

array([0, 4, 1, 3, 2, 7, 6, 5, 9], dtype=int64)

In [14]:
scaler = MinMaxScaler()
spiciness_2d = data_add_dummies['spiciness'].values.reshape(-1, 1)
difficulty_2d = data_add_dummies['difficulty'].values.reshape(-1, 1)


scaled_data = data_add_dummies
scaled_data['spiciness'] = scaler.fit_transform(spiciness_2d)
scaled_data['difficulty'] = scaler.fit_transform(difficulty_2d)



In [15]:
pd.unique(data['spiciness'])

array([0, 4, 1, 3, 2, 7, 6, 5, 9], dtype=int64)

In [16]:
data[data['spiciness'] == 4].head(10)

,메뉴 이름,방법 분류,국가 분류,테마 분류,난이도 분류,칼로리,조회수,분량,주재료 이름,부재료 이름,양념,레시피,조리시간,main_ingredients,sub_ingredients,seasoning,whole_ingredients,spiciness,low_calorie,difficulty
2,김치볶음밥,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,560.0,337639,2,"['김치(잘게썬것) 1/3포기', '밥 2공기']","['감자 1/4개', '양파 1/2개', '햄 60g', '식용유 약간', '계란 2개']","['소금 약간', '후춧가루 약간']","['1. 김치는 1cm 크기로 썰고 햄, 양파, 감자도 1cm 정도의 크기로 잘게 썬다.', '2. 프라이팬에 기름을 두르고 김치와 감자를 볶다가 햄과 양파를 넣고 볶아준다.', '3. 양파가 어느 정도 익으면 밥을 넣고 펴듯이 볶아준다.', '4. 소금과 후춧가루로 간을 맞추고 계란프라이를 얹어 낸다.']",20,"[김치, 밥]","[감자, 양파, 햄, 식용유, 계란]","[소금, 후춧가루]","[김치, 밥, 감자, 양파, 햄, 식용유, 계란, 소금, 후춧가루]",4,0,1
20,쇠고기김치말이,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,352.0,43108,2,"['쇠고기 400g', '밥 1공기', '김치 200g']","['양파 30g', '당근 30g', '피망 20g', '검은깨 약간']","['소금 약간', '후춧가루 약간']","['1. 쇠고기는 곱게 다져서 밑간을 약하게 한 후 끈기있게 치댄다.', '2. 김치는 속을 털어 내 물기를 살짝 짜내고 양파, 당근, 피망은 다진다.', '3. 팬에 식용유를 두르고 양파, 당근을 볶다가 밥을 넣어 계속 볶은 뒤 피망과 검은깨를 넣고 소금, 후춧가루로 간을 맞춘다.', '4. 알루미늄 호일에 쇠고기를 깔고 그 위에 김치를 깐다.', '5. 볶은 밥을 ④에 올려 놓고 말아 준 뒤 알루미늄 호일에 싼다.', '6. 오븐 팬에 ⑤를 놓고 230℃ 정도에서 20분간 익힌다.']",30,"[쇠고기, 밥, 김치]","[양파, 당근, 피망, 검은깨]","[소금, 후춧가루]","[쇠고기, 밥, 김치, 양파, 당근, 피망, 검은깨, 소금, 후춧가루]",4,1,1
21,쇠고기김치말이밥,"['밥', '죽', '스프']",한식,메인요리,매우 쉬움,432.0,16356,2,"['쇠고기(안심) 300g', '찬밥 1/2공기', '김치 50g']","['양파 1/4개', '피망 1/4개', '당근 1/4개']","['소금 1/2작은술', '후춧가루 1/4작은술']","['1. 쇠고기는 키친타올로 핏물을 제거한 후 18cm x 15cm, 두께 0.5cm로 저며 소금, 후춧가루로 밑간한다.', '2. 김치는 물기를 제거해 다지고 양파, 피망, 당근도 곱게 다진다.', '3. 프라이팬에 기름을 두르고 찬밥을 볶다가 김치, 양파, 피망, 당근을 넣어 가볍게 볶은 후 소금, 후춧가루로 간한다.', '4. 김발 위에 쇠고기를 편 후 준비한 ③을 올리고 돌돌 말아준다.', '5. 190℃로 예열한 오븐에 25~30분간 구워낸 후 한 김 식혀 잘라 접시에 담는다.']",30,"[쇠고기, 찬밥, 김치]","[양파, 피망, 당근]","[소금, 후춧가루]","[쇠고기, 찬밥, 김치, 양파, 피망, 당근, 소금, 후춧가루]",4,1,1
86,갈치조림,['조림'],한식,메인요리,매우 쉬움,240.0,148088,2,['갈치 4토막'],"['무 1/3개', '애호박 1/2개', '당근 1/4개', '대파 1개', '양파 1/2개', '홍고추 1개', '청고추 1개']","['굵은고춧가루 2큰술', '고운고춧가루 1큰술', '설탕 1큰술', '다진생강 2작은술', '다진마늘 1큰술', '정종 1큰술', '된장 2작은술', '후춧가루 약간', '소금 약간', '간장 2큰술', '물엿 2큰술']","['1. 갈치를 깨끗이 손질하여 알맞은 크기로 잘라 놓는다.', '2. 채소는 길이 4~5cm, 두께 2.5cm 정도로 얇게 썰어 보기 좋게 놓는다.', '3. 그 위에 생선을 얹고 양념장을 뿌려 중, 약불에 끓인다.', '4. 끓인 후, 홍고추와 청고추로 장식한다.']",30,[갈치],"[무, 애호박, 당근, 대파, 양파, 홍고추, 청고추]","[굵은고춧가루, 고운고춧가루, 설탕, 다진생강, 다진마늘, 정종, 된장, 후춧가루, 소금, 간장, 물엿]","[갈치, 무, 애호박, 당근, 대파, 양파, 홍고추, 청고추, 굵은고춧가루, 고운고춧가루, 설탕, 다진생강, 다진마늘, 정종, 된장, 후춧가루, 소금, 간장, 물엿]",4,1,1
91,고등어조림,['조림'],한식,메인요리,매우 쉬움,250.0,215992,2,['고등어 1마리'],"['무 1/3개', '애호박 1/2개', '당근 1/4개', '대파 1개', '양파 1/2개', '홍고추 2개', '풋고추 2개', '팽이버섯 1/2팩']","['간장 2큰술', '굵은고춧가루 2큰술', '고운고춧가루 1큰술', '설탕 1큰술', '물엿 2큰술', '생강 2작은술', '마늘 1큰술', '정종 1큰술', '된장 2큰술', '후춧가루 약간', '소금 약간']","['1. 고등어를 깨끗이 손질하여 4~5cm 정도 알맞은 크기로 잘라 놓는다.', '2. 채소는 4~5cm, 두께 2.5cm 정도로 얇게 썰어 보기 좋게 놓는다.', '3. 그 위에 생선을 얹고 양념장을 뿌려 중, 약불에 끓인다.', '4. 끓인 후, 홍고추, 청고추와 팽이버섯으로 고명한다.']",30,[고등어],"[무, 애호박, 당근, 대파, 양파, 홍고추, 풋고추, 팽이버섯]","[간장, 굵은고춧가루, 고운고춧가루, 설탕, 물엿, 생강, 마늘, 정종, 된장, 후춧가루, 소금]","[고등어, 무, 애호박, 당근, 대파, 양파, 홍고추, 풋고추, 팽이버섯, 간장, 굵은고춧가루, 고운고춧가루, 설탕, 물엿, 생강, 마늘, 정종, 된장, 후춧가루, 소금]",4,1,1
93,굴비조림,['조림'],한식,메인요리,매우 쉬움,254.0,37890,2,['굴비 2마리'],"['무 1/3개', '애호박 1/2개', '당근 1/4개', '대파 1개', '양파 1/2개', '홍고추 1개', '청고추 1개', '팽이버섯 1/2팩', '굵은고춧가루 2큰술', '고운고춧가루 1큰술']","['설탕 1큰술', '물엿 2큰술', '생강 2작은술', '마늘 1큰술', '정종 1큰술', '된장 2작은술', '후춧가루 약간', '소금 약간']","['1. 굴비를 깨끗이 손질하여 알맞은 크기로 잘라 놓는다.', '2. 채소는 4~5cm, 두께 2.5cm 정도로 얇게 썰어 보기 좋게 놓는다.', '3. 그 위에 생선을 얹고 양념장을 뿌려 중불이나 약불에 끓인다.', '4. 홍고추, 청고추와 팽이버섯을 얹고 살짝 한번 더 끓인다.']",30,[굴비],"[무, 애호박, 당근, 대파, 양파, 홍고추, 청고추, 팽이버섯, 굵은고춧가루, 고운고춧가루]","[설탕, 물엿, 생강, 마늘, 정종, 된장, 후춧가루, 소금]","[굴비, 무, 애호박, 당근, 대파, 양파, 홍고추, 청고추, 팽이버섯, 굵은고춧가루, 고운고춧가루, 설탕, 물엿, 생강, 마늘, 정종, 된장, 후춧가루, 소금]",4,1,1
110,고등어양념구이,['구이'],한식,메인요리,매우 쉬움,175.0,49723,2,['고등어 1/2마리'],"['대파 1개', '양파 1개', '실파 3개', '홍고추 2개', '청고추 2개']","['간장 3큰술', '굵은고춧가루 2큰술', '고운고춧가루 1큰술', '정종 2큰술', '생강 1개', '마늘 5쪽', '참기름 2작은술', '깨소금 1큰술', '물엿 2큰술']","['1. 고등어를 머리, 꼬리와 내장을 제거하고 손질한 후, 포를 떠서 뼈를 발라낸다.', '2. 양념장을 만들어 고등어를 12시간 재운 후 석쇠에 굽는다.', '3. 구울 때 양념장을 발라 가면서 굽는다.', '4. 센 불에 구우면 생선이 완전히 익지 않으므로 중, 약불에 구워낸다.']",30,[고등어],"[대파, 양파, 실파, 홍고추, 청고추]","[간장, 굵은고춧가루, 고운고춧가루, 정종, 생강, 마늘, 참기름, 깨소금,

In [17]:
scaled_data.head()

,메뉴 이름,난이도 분류,칼로리,조리시간,main_ingredients,sub_ingredients,seasoning,whole_ingredients,spiciness,low_calorie,difficulty,is_southeast_asian,is_mexican,is_western,is_italian,is_japanese,is_chinese,is_mixed,is_korean,is_snack,is_main,is_side
0,견과류 영양솥밥,쉬움,311.0,60,"[백미, 은행, 밤, 대추, 잣]",[물],[],"[백미, 은행, 밤, 대추, 잣, 물]",0.000000,1,0.333333,False,False,False,False,False,False,False,True,False,True,False
1,굴밥,매우 쉬움,320.0,30,"[굴, 쌀, 보리쌀]","[어묵, 표고버섯, 당근, 완두콩, 물]",[소금],"[굴, 쌀, 보리쌀, 어묵, 표고버섯, 당근, 완두콩, 물, 소금]",0.000000,1,0.000000,False,False,False,False,False,False,False,True,False,True,False
2,김치볶음밥,매우 쉬움,560.0,20,"[김치, 밥]","[감자, 양파, 햄, 식용유, 계란]","[소금, 후춧가루]","[김치, 밥, 감자, 양파, 햄, 식용유, 계란, 소금, 후춧가루]",0.444444,0,0.000000,False,False,False,False,False,False,False,True,False,True,False
3,꽃상추쌈,매우 쉬움,352.0,30,"[상추, 밥]","[다진쇠고기, 다진양파]","[간장, 설탕, 다진파, 다진마늘, 깨소금, 참기름, 고추장]","[상추, 밥, 다진쇠고기, 다진양파, 간장, 설탕, 다진파, 다진마늘, 깨소금, 참기름, 고추장]",0.111111,1,0.000000,False,False,False,False,False,False,False,True,False,True,False
4,녹두죽,매우 쉬움,280.0,60,"[녹두, 쌀]",[물],"[소금, 설탕]","[녹두, 쌀, 물, 소금, 설탕]",0.000000,1,0.000000,False,False,False,False,False,False,False,True,False,True,False


## 사용자 정보 기반 필터링 및 코사인 유사도

In [18]:
allergic_ingredients = ['계란', '달걀'] # 알레르기 반응 있는 식품 예시
max_cooking_time = 30

In [45]:
user_ingredients = ['참나물', '고추장', '고추가루', '굴', '쌀', '대추', '보리쌀', '콩', '생강', '마늘', '현미', '잣', '호두', '아몬드', '고추', '참깨', '된장', '쌀국수', '바지락', '파프리카', '숙주', '식용유', '다진마늘', '말린고추', '해선장', '굴소스']
market_ingredients = ['오징어', '굴', '쌀', '대추', '보리쌀', '콩', '생강', '마늘', '현미', '잣', '호두', '아몬드', '고추', '참깨', '된장', '쌀국수', '바지락', '파프리카', '숙주', '식용유', '다진마늘', '말린고추', '해선장', '굴소스']

In [46]:
# 사용자 데이터 정의
user_data = {
    'spiciness': [1], # (0, 0.2, 0.4, 0.6, 0.8, 1) 클 수록 매운 음식을 선호하는 것
    'difficulty': [0.4],
    'low_calorie': [1],
    'is_southeast_asian': [0],
    'is_mexican': [1],
    'is_western': [0.5],
    'is_italian': [0.5],
    'is_japanese': [0.7],
    'is_chinese': [0],
    'is_mixed': [0],
    'is_korean': [0],
    'is_snack': [0],
    'is_main': [0],
    'is_side': [0]
}

user_df = pd.DataFrame(user_data)

user_data2 = user_df.drop(columns=['spiciness'])

user_data3 = user_df.drop(columns=['difficulty'])

user_data4 = user_df.drop(columns=['low_calorie'])

user_data5 = user_df.drop(columns=['is_southeast_asian', 'is_western', 'is_italian', 'is_japanese', 'is_chinese', 'is_mixed', 'is_korean'])


# 매움 정보 제거한 데이터프레임
user_df2 = pd.DataFrame(user_data2)
# 요리 난이도 정보 제거한 데이터프레임
user_df3 = pd.DataFrame(user_data3)
# 칼로리 정보 제거한 데이터프레임
user_df4 = pd.DataFrame(user_data4)
# 나라 정보 제거한 데이터프레임
user_df5 = pd.DataFrame(user_data5)

In [47]:
def exclude_allergic_ingredients(row, allergic_ingredients):
    return not any(ingredient in row['whole_ingredients'] for ingredient in allergic_ingredients)

In [48]:
# 필터링
recommendation_df = scaled_data[scaled_data.apply(exclude_allergic_ingredients, axis=1, allergic_ingredients=allergic_ingredients)]
recommendation_df = recommendation_df[recommendation_df['조리시간'] < max_cooking_time] 

In [49]:
recommendation_df.head()

,메뉴 이름,난이도 분류,칼로리,조리시간,main_ingredients,sub_ingredients,seasoning,whole_ingredients,spiciness,low_calorie,difficulty,is_southeast_asian,is_mexican,is_western,is_italian,is_japanese,is_chinese,is_mixed,is_korean,is_snack,is_main,is_side
9,대구계란죽,매우 쉬움,260.0,20,"[대구, 쌀]","[계란노른자, 대파, 물]",[멸치],"[대구, 쌀, 계란노른자, 대파, 물, 멸치]",0.000000,1,0.0,False,False,False,False,False,False,False,True,False,True,False
30,우렁된장쌈밥,매우 쉬움,350.0,20,[우렁],"[호박, 양파, 고추, 대파, 밥]","[된장, 고춧가루, 국멸치]","[우렁, 호박, 양파, 고추, 대파, 밥, 된장, 고춧가루, 국멸치]",0.222222,1,0.0,False,False,False,False,False,False,False,True,False,True,False
32,웰빙비빔밥,매우 쉬움,412.0,20,[보리밥],"[콩나물, 부추, 김가루, 다진양파]","[소금, 들기름, 쌈장, 고추장, 고춧가루, 식초, 설탕, 간장, 청양고추, 다진마늘]","[보리밥, 콩나물, 부추, 김가루, 다진양파, 소금, 들기름, 쌈장, 고추장, 고춧가루, 식초, 설탕, 간장, 청양고추, 다진마늘]",0.333333,1,0.0,False,False,False,False,False,False,False,True,False,True,False
55,다시마냉국,매우 쉬움,63.0,20,[다시마],"[오이, 홍고추, 물]","[다진파, 다진마늘, 소금, 고춧가루, 깨소금]","[다시마, 오이, 홍고추, 물, 다진파, 다진마늘, 소금, 고춧가루, 깨소금]",0.222222,1,0.0,False,False,False,False,False,False,False,True,False,True,False
75,오이냉국,매우 쉬움,82.0,20,"[오이, 물]","[대파, 홍고추, 물]","[파, 다진마늘, 간장, 식초, 설탕, 깨소금, 고춧가루, 소금]","[오이, 물, 대파, 홍고추, 물, 파, 다진마늘, 간장, 식초, 설탕, 깨소금, 고춧가루, 소금]",0.222222,1,0.0,False,False,False,False,False,False,False,True,False,True,False


### 추천시스템 A


In [50]:
def contains_all_ingredients(row, user_ingredients):
    return set(row['whole_ingredients']).issubset(set(user_ingredients))

recommendation_A_list = recommendation_df[recommendation_df.apply(contains_all_ingredients, axis=1, user_ingredients=user_ingredients)]
recommendation_A_list[['메뉴 이름', 'whole_ingredients']]

,메뉴 이름,whole_ingredients


In [51]:
def filter_recipes(row, user_ingredients, market_ingredients, max_additional_items=5):
    recipe_ingredients = set(row['whole_ingredients'])
    
    user_ingredients_set = set(user_ingredients)
    market_ingredients_set = set(market_ingredients)
    
    # 레시피에서 부족한 재료들 (마켓이든 어디서든 추가로 구매해야함)
    missing_ingredients = recipe_ingredients - user_ingredients_set
    
    
    # 부족한 재료 중 마켓에서 구매 가능한 재료
    purchasable_items_in_market = missing_ingredients & market_ingredients_set

    # 부족한 재료 중 마켓에서 구하지 못하는 재료 (purchasable_ingredients_in_market의 여집합)
    additional_items_needed = missing_ingredients - market_ingredients_set

    # 마켓이든 어디든 추가로 구매해야 하는 재료의 수가 max_additional_items 이하인 경우만 True
    can_make = len(missing_ingredients) <= max_additional_items

    return can_make, missing_ingredients, purchasable_items_in_market, additional_items_needed


def count_matching_ingredients(row, user_ingredients):
    recipe_ingredients = set(row['whole_ingredients'])
    matching_ingredients_count = len(recipe_ingredients & set(user_ingredients))
    return matching_ingredients_count


def count_missing_ingredients(row, user_ingredients):
    matching_ingredients_count = len(set(row['whole_ingredients']))
    # matching_ingredients_count = len(recipe_ingredients & set(user_ingredients))
    return matching_ingredients_count


# 필터링된 데이터프레임 생성
recommendation_df['matching_count'] = recommendation_df.apply(count_matching_ingredients, axis=1, user_ingredients=user_ingredients)
recommendation_df['missing_count'] = recommendation_df.apply(count_missing_ingredients, axis=1, user_ingredients=user_ingredients)

recommendation_df['can_make'], recommendation_df['missing_ingredients'], recommendation_df['purchasable_items_in_market'], recommendation_df['additional_items_needed']= zip(*recommendation_df.apply(filter_recipes, axis=1, user_ingredients=user_ingredients, market_ingredients=market_ingredients, max_additional_items=5))
recommendation_B_list = recommendation_df[recommendation_df['can_make']==True]

# 그 중 사용자가 가지고 있는 재료와 가장 겹치는게 많되, 필요한 재료는 적은 레시피 Top 20
recommendation_B_list = recommendation_B_list.sort_values(by='matching_count', ascending=False)[0:20]


recommendation_B_list = recommendation_B_list.sort_values(
    by=['matching_count', 'missing_count'],
    ascending=[False, True]
)

# 상위 20개 행 선택
recommendation_B_list = recommendation_B_list.head(20)

## 코사인 유사도

In [52]:
weights = {
    'spiciness': 1.5,
    'difficulty': 0.8,
    'low_calorie': 0.5,
    'is_southeast_asian': 0.2,
    'is_mexican': 0.2,
    'is_western': 0.2,
    'is_italian': 0.2,
    'is_japanese': 0.2,
    'is_chinese': 0.2,
    'is_mixed': 0.2,
    'is_korean': 0.2,
    'is_snack': 1.0,
    'is_main': 1.0,
    'is_side': 1.0
}



recipe_df = recommendation_df[['spiciness',
                              'difficulty',
                              'low_calorie',
                              'is_southeast_asian',
                              'is_mexican',
                              'is_western',
                              'is_italian',
                              'is_japanese',
                              'is_chinese',
                              'is_mixed',
                              'is_korean',
                              'is_snack',
                              'is_main',
                              'is_side'
                              ]]

recipe_df2 = recipe_df.drop(columns=['spiciness'])

recipe_df3 = recipe_df.drop(columns=['difficulty'])

recipe_df4 = recipe_df.drop(columns=['low_calorie'])

recipe_df5 = recipe_df.drop(columns=['is_southeast_asian', 'is_western', 'is_italian', 'is_japanese', 'is_chinese', 'is_mixed', 'is_korean'])


new_series = pd.Series(['사용자'])

updated_series = pd.concat([new_series, recommendation_df['메뉴 이름']], ignore_index=True)


# 사용자 데이터프레임과 기존 데이터프레임을 합쳐서 유사도 계산
combined_df = pd.concat([user_df, recipe_df], ignore_index=True)
combined_df2 = pd.concat([user_df2, recipe_df2], ignore_index=True)
combined_df3 = pd.concat([user_df3, recipe_df3], ignore_index=True)
combined_df4 = pd.concat([user_df4, recipe_df4], ignore_index=True)
combined_df5 = pd.concat([user_df5, recipe_df5], ignore_index=True)


def apply_weights(df, weights):
    for column in df.columns:
        if column in weights:  # 딕셔너리에 컬럼이 존재하는지 확인
            df[column] = df[column] * weights[column]  # 가중치 적용
    return df

# 각 DataFrame에 가중치 적용
combined_df = apply_weights(combined_df, weights)
combined_df2 = apply_weights(combined_df2, weights)
combined_df3 = apply_weights(combined_df3, weights)
combined_df4 = apply_weights(combined_df4, weights)
combined_df5 = apply_weights(combined_df5, weights)

# 코사인 유사도 계산
similarity_matrix = cosine_similarity(combined_df)
similarity_matrix2 = cosine_similarity(combined_df2)
similarity_matrix3 = cosine_similarity(combined_df3)
similarity_matrix4 = cosine_similarity(combined_df4)
similarity_matrix5 = cosine_similarity(combined_df5)

# # 사용자 데이터와 기존 데이터 간의 유사도 출력
similarity_df = pd.DataFrame(similarity_matrix, index=updated_series, columns=updated_series)
similarity_df2 = pd.DataFrame(similarity_matrix2, index=updated_series, columns=updated_series)
similarity_df3 = pd.DataFrame(similarity_matrix3, index=updated_series, columns=updated_series)
similarity_df4 = pd.DataFrame(similarity_matrix4, index=updated_series, columns=updated_series)
similarity_df5 = pd.DataFrame(similarity_matrix5, index=updated_series, columns=updated_series)

In [53]:
similarity_df.head(5)

,사용자,대구계란죽,우렁된장쌈밥,웰빙비빔밥,다시마냉국,오이냉국,우무콩냉국,콩가루에빠진쑥국,콩나물냉국,견과류조림,두부양념조림,가자미구이,갈치구이,건포도갈비구이,김구이,꽁치구이,대파미소구이,새송이버섯구이,자반고등어구이,잡채전병튀김,꼬막양념찜,명란찜,쇠고기가지찜,대파생채,도라지생채,도라지오이생채,도토리묵무침,돌나물달래샐러드,무생채,묵과양념장,미나리꼬막간장무침,상추채소무침,숙주미나리무침,시금치무침,오이나물,우묵냉채,참나물간장무침,참나물고추장무침,채소무침,팽이버섯냉채,깻잎조림,된장깻잎장아찌,마늘장아찌,마늘쫑굴소스볶음,무말랭이장아찌,시금치나물,콩나물무침,콩나물미나리무침,파래무침,향신양념무장아찌,향신양념콩나물무침,쌈장,배숙,수박화채,오과차,고추장얼큰라면,바지락칼국수,오징어무침냉면,가지쇠고기볶음,고구마줄기볶음,고사리볶음,두부김치,마른새우볶음,멸치꽈리고추볶음,멸치볶음,버섯볶음과들깨소스,불고기,순대볶음,연근볶음,오징어채볶음,잔멸치쌈장볶음,모듬콩수프,속채운감자,콘버터,콘치즈구이,홍합구이,단호박크림찜,토마토와찐갈치,과일샐러드,과일샐러드,그린샐러드,레이즌컨츄리샐러드,모듬샐러드,바질토마토두부샐러드,사과치즈샐러드,시금치샐러드,시저샐러드,양상추참치샐러드,연어샐러드,치즈아보카도,치즈토마토,콘샐러드,키위드레싱과콘샐러드,토마토샐러드,흑미 샐러드,채소피클,가리비그라탕,계피향의 바나나구이,모듬과일,시드니신풀선데,싱가폴슬링,장미꽃아이스크림,마요네즈드레싱,애플소스,타르타르소스,마늘빵,초콜릿퐁듀,치즈카나페,호일초콜릿,까페라떼,딸기스무디,딸기스무디,레몬에이드,롱아일랜드아이스티,모스코바뮬,발효초요구르트,아메리카노,아이스커피,아이스후르츠소다,카페라떼,키위주스,토마토주스,플랜터즈,터널바게트,핫도그샌드위치,모듬채소볶음,베이비콘채소볶음,소시지채소케첩볶음,상추미소국,갈치채소말이조림,문어초회,해초무침,와사비간장,굴소스와 튀김갈치,마늘콩소스의해산물요리,은대구두치소스,드라이카레볶음밥,피넛소스와밥,쌀국수냉채,타이식 오이땅콩 샐러드,피넛버터소스,망고라씨,망고시미루,Che Thack Luu(채택루),닭가슴살 샐러드와 오렌지드레싱,로메인 시저샐러드,모짜렐라 카프레제,조개샐러드,브루스케타,페델리니 카르보나라,버섯파스타,시칠리안 링귀니,시금치베이컨주먹밥,이유식치즈죽,채소치즈죽,치즈멸치볼,케첩볶음밥,치즈국,단호박구이,대파미트볼꼬치,치즈감자칩,버섯호일찜,냉고기쌈,된장소스와 채소샐러드,배깨초무침,부추올리브오일무침,수삼치즈샐러드,어묵샐러드,와사비크림치즈와채소스틱,참치샐러리스틱,참치회무침,참치회무침,피스타치오샐러드,자장떡볶이,과일 깍기,초코밤범벅,유자드레싱,과일찰떡카나페,감주스,귤주스,딸기주스,바나나주스,배주스,사과주스,생토마토주스,요구르트두부주스,자두주스,과일천사채국수,명란스파게티,치즈볶음라면,연근카레전,대파 파프리카굴소스볶음,삶은무해산물볶음
사용자,1.000000,0.134405,0.386897,0.492051,0.386897,0.386897,0.134405,0.265962,0.265962,0.134405,0.386897,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.492051,0.134405,0.134405,0.265962,0.386897,0.386897,0.134405,0.265962,0.265962,0.579563,0.265962,0.265962,0.265962,0.386897,0.265962,0.265962,0.134405,0.265962,0.265962,0.134405,0.579563,0.265962,0.134405,0.386897,0.134405,0.134405,0.134405,0.492051,0.265962,0.265962,0.265962,0.134405,0.265962,0.265962,0.386897,0.134405,0.134405,0.134405,0.811351,0.134405,0.386897,0.386897,0.134405,0.134405,0.579563,0.134405,0.579563,0.265962,0.134405,0.134405,0.265962,0.265962,0.265962,0.134405,0.145158,0.145158,0.145158,0.145158,0.397214,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.149458,0.149458,0.149458,0.149458,0.149458,0.265962,0.134405,0.386897,0.134405,0.000000,0.265962,0.134405,0.134405,0.134405,0.134405,0.134405,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.145158,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.265962,0.265962,0.134405,0.265962,0.134405,0.134405,0.134405,0.134405,0.492051,0.386897,0.134405,0.386897,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.134405,0.265962
대구계란죽,0.134405,1.000000,0.959530,0.915239,0.959530,0.959530,1.000000,0.989404,0.989404,0.224806,0.215708,1.000000,1.000000,1.000000,0.224806,1.000000,1.000000,0.915239,1.000000,1.000000,0.989404,0.959530,0.959530,0.224806,0.222424,0.222424,0.193876,0.222424,0.222424,0.222424,0.215708,0.222424,0.222424,0.224806,0.222424,0.989404,0.224806,0.193876,0.222424,1.000000,0.215708,0.224806,0.224806,0.224806,0.205751,0.222424,0.222424,0.222424,0.224806,0.222424,0.222424,0.215708,0.224806,0.224806,0.224806,0.603661,1.000000,0.959530,0.959530,1.000000,1.000000,0.862412,0.224806,0.193876,0.222424,0.224806,1.000000,0.989404,0.222424,0.222424,0.224806,0.968992,0.968992,0.968992,0.968992,0.929777,0.968992,0.968992,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.193798,0.968992,0.193798,0.193798,0.193798,0.19379

In [54]:
def get_top_similar_recipes(similarity_df, recipe_name, top_n=5):
    # 특정 메뉴에 대한 유사도 값 추출 및 정렬
    similar_scores = similarity_df[recipe_name].sort_values(ascending=False)
    # 상위 top_n 메뉴 추출 (자기 자신 제외)
    top_similar = similar_scores.iloc[1:top_n+1]
    return top_similar

In [55]:
# 예시: 'recipe1'에 대해 가장 높은 유사도 값을 갖는 top 3 메뉴 추출
top_10_similar = get_top_similar_recipes(similarity_df, '사용자', top_n=10)
print(top_10_similar)

고추장얼큰라면     0.811351
참나물고추장무침    0.579563
도토리묵무침      0.579563
두부김치        0.579563
멸치꽈리고추볶음    0.579563
참치회무침       0.492051
새송이버섯구이     0.492051
무말랭이장아찌     0.492051
웰빙비빔밥       0.492051
홍합구이        0.397214
Name: 사용자, dtype: float64


In [56]:
top_10_similar2 = get_top_similar_recipes(similarity_df2, '사용자', top_n=10)
print(top_10_similar2)

해초무침          0.372399
상추미소국         0.372399
갈치채소말이조림      0.372399
문어초회          0.372399
와사비간장         0.372399
모듬과일          0.361682
계피향의 바나나구이    0.361682
채소피클          0.361682
흑미 샐러드        0.361682
양상추참치샐러드      0.361682
Name: 사용자, dtype: float64


In [59]:
from openai import OpenAI
client = OpenAI(api_key = 'YOUR API KEY')
def recommend_menu(user_context, top_similar_recipes):
    # LLM을 사용하여 10개의 추천 메뉴 중에서 3개의 메뉴를 선택하는 함수
    prompt = f"""
    You are a smart recipe recommender. Based on the user's context below, choose the 3 most suitable recipes from the given list. You have to consider user's mood and explain reasonable reason.

    User context: {user_context}

    Recipes:
    {', '.join(top_similar_recipes.index)}

    Provide the names of the 3 most suitable recipes and explain why you recommend each of them.
    Preferably, the number of 'items users should buy in market' should be greater than the number of 'additional items'.

    Format the response as:
    Recipe: <recipe name>
    Reason: <reason in Korean>
    items that users should buy in market: <purchasable_items_in_market>
    additional items: <additional_items_needed>

    """


    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=800,
        temperature=0.7
    )
    response_text = response.choices[0].text.strip()
    recommendations = []
    items = response_text.split('\n\n')
    for item in items:
        if 'Recipe:' in item and 'Reason:' in item:
            recipe = item.split('Recipe:')[1].split('Reason:')[0].strip()
            reason = item.split('Reason:')[1].strip()
            recommendations.append((recipe, reason))

    return recommendations[:3]

    


In [61]:
# 사용자 상황 예시
user_context = "오늘 아침에는 토스트를 먹었고, 커피를 두잔 마셨어. 지금은 좀 피곤해서 요리에 많은 시간을 쏟고 싶지 않고, 저녁에는 치킨과 맥주를 먹을 예정이야"

# 메뉴 추천
recommended_menus = recommend_menu(user_context, top_10_similar)

for recipe, reason in recommended_menus:
    print(f"Recommended Menu: {recipe}\nReason: {reason}\n")
    

Recommended Menu: 두부김치
Reason: 사용자가 토스트와 커피를 먹었기 때문에 아침에는 이미 간단한 음식을 먹었을 것이다. 따라서 점심에는 무겁게 먹지 않을 것이고, 두부김치는 가볍게 먹을 수 있는 음식이다. 또한 무침 요리이기 때문에 요리 시간도 짧게 걸릴 것이다.
items that users should buy in market: 두부, 배추김치, 고춧가루
additional items: 없음
used items: 두부, 배추김치, 고춧가루

Recommended Menu: 멸치꽈리고추볶음
Reason: 사용자가 지금 피곤해서 요리에 많은 시간을 쏟고 싶지 않기 때문에 간단하게 만들 수 있는 음식이 적합하다. 또한 고추장 얼큰라면과 비슷한 매운맛의 요리이기 때문에 사용자의 입맛에도 맞을 것이다.
items that users should buy in market: 멸치, 꽈리고추, 고춧가루
additional items: 없음
used items: 멸치, 꽈리고추, 고춧가루

Recommended Menu: 참치회무침
Reason: 사용자가 저녁에 치킨과 맥주를 먹을 예정이기 때문에 점심에는 건강한 음식을 먹으면 좋을 것이다. 참치회무침은 가볍게 먹을 수 있지만 영양가가 높은 요리이다. 또한 무침 요리이기 때문에 요리 시간도 짧게 걸릴 것이다.
items that users should buy in market: 참치, 고춧가루, 소금, 식초
additional items: 없음
used items: 참치, 고춧가루, 소금, 식초

